# Семинар_Оценка качества RAG


Максимально близко к [оригиналу](https://huggingface.co/learn/cookbook/en/rag_evaluation)  by: [Aymeric Roucher](https://huggingface.co/m-ric)

Здесь мы снова создадим датасет с вопросами и ответами и затем применим агента [LLM-as-a-judge](https://huggingface.co/papers/2306.05685)


In [ ]:
!pip install -q torch transformers langchain sentence-transformers tqdm openpyxl openai pandas datasets langchain-community ragatouille

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 59.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 39.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 26.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 6.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 13.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 6.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 6.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 80.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 491.2/491.2 kB 32.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 82.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1

In [ ]:
%reload_ext autoreload
%autoreload 2

In [ ]:
from tqdm.auto import tqdm
import pandas as pd
from typing import Optional, List, Tuple
import json
import datasets

pd.set_option("display.max_colwidth", None)

Здесь понадобится ваш токен от HF

In [ ]:
#
from huggingface_hub import notebook_login

notebook_login()

### Load your knowledge base

In [ ]:
ds = datasets.load_dataset("m-ric/huggingface_doc", split="train")

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


README.md:   0%|          | 0.00/21.0 [00:00<?, ?B/s]

huggingface_doc.csv:   0%|          | 0.00/22.0M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/2647 [00:00<?, ? examples/s]

# 1. Синтетический датасет для теста
We first build a synthetic dataset of questions and associated contexts. The method is to get elements from our knowledge base, and ask an LLM to generate questions based on these documents.

Then we setup other LLM agents to act as quality filters for the generated QA couples: each of them will act as the filter for a specific flaw.

### 1.1. Готовим документы

In [ ]:
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.docstore.document import Document as LangchainDocument

langchain_docs = [
    LangchainDocument(page_content=doc["text"], metadata={"source": doc["source"]})
    for doc in tqdm(ds)
]


text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=2000,
    chunk_overlap=200,
    add_start_index=True,
    separators=["\n\n", "\n", ".", " ", ""],
)

docs_processed = []
for doc in langchain_docs:
    docs_processed += text_splitter.split_documents([doc])

  0%|          | 0/2647 [00:00<?, ?it/s]

### 1.2. LLM

в ячейке ниже вместо метода .post() пришлось вызывать таску напрямую .text_generation()

In [ ]:
from huggingface_hub import InferenceClient


repo_id = "mistralai/Mixtral-8x7B-Instruct-v0.1" ## пример про прогерские вопросы

llm_client = InferenceClient(
    model=repo_id
    , timeout=120
)

def call_llm(inference_client: InferenceClient, prompt: str):
    response = inference_client.text_generation(
        prompt=prompt,
        max_new_tokens=1000,
        temperature=1.0,  # optional
        # top_p=0.95,       # optional
        do_sample=True    # optional
    )
    return response
    return json.loads(response.decode())[0]["generated_text"]


call_llm(llm_client, "This is a test context")

' or test root. Mocha looks for these if configured to on the root level.\n\nEnzyme is configured for React 15 / 16 in node\\_modules/jest-enzyme. It\nexpects us to always import React when rendering otherwise it attempts to\naccept `React` as a prop.\n\n# Quick "One Off" Testing\nClone this into an empty project folder then run\n\n    yarn && npm test\n\nto start up test and coverage watchers.\n\nThis will re-run tests any time you save a modified file. It will fail on the\nfirst incorrect test and only run tests related to the file that you saved\nedits in.\n\nThis watches via Jest with the `--watchAll` option. This means that you\'ll have\ntwo separate watchers in two windows. The Jest output window is benign, large,\nand not designed for interaction. You can hide the window (unless you are in the\n_coverage_ directory and then only with [Chrome](file:///Users/cscheuermann/codebase/10.maybe/ldap-app/README.md#coverage)) after you get the Jest logo.\n\nYou can still alter compiler op

In [ ]:
QA_generation_prompt = """
Your task is to write a factoid question and an answer given a context.
Your factoid question should be answerable with a specific, concise piece of factual information from the context.
Your factoid question should be formulated in the same style as questions users could ask in a search engine.
This means that your factoid question MUST NOT mention something like "according to the passage" or "context".

Provide your answer as follows:

Output:::
Factoid question: (your factoid question)
Answer: (your answer to the factoid question)

Now here is the context.

Context: {context}\n
Output:::"""

Попробуем сгенерить всего лишь 10 вопросно-ответных пар, и загрузим остальные с Hub.

Здесь автор дает такую рекомендацию:



> But for your specific knowledge base, given that you want to get at least ~100 test samples, and accounting for the fact that we will filter out around half of these with our critique agents later on, you should generate much more, in the >200 samples.



Что ж, запасайтесь токенами (или разворачивайте LLM локально через Ollama / vllm)

In [ ]:
import random

N_GENERATIONS = 10  # We intentionally generate only 10 QA couples here for cost and time considerations

print(f"Generating {N_GENERATIONS} QA couples...")

outputs = []
for sampled_context in tqdm(random.sample(docs_processed, N_GENERATIONS)):
    # Generate QA couple
    output_QA_couple = call_llm(
        llm_client, QA_generation_prompt.format(context=sampled_context.page_content)
    )
    try:
        question = output_QA_couple.split("Factoid question: ")[-1].split("Answer: ")[0]
        answer = output_QA_couple.split("Answer: ")[-1]
        assert len(answer) < 300, "Answer is too long"
        outputs.append(
            {
                "context": sampled_context.page_content,
                "question": question,
                "answer": answer,
                "source_doc": sampled_context.metadata["source"],
            }
        )
    except:
        continue

Generating 10 QA couples...


  0%|          | 0/10 [00:00<?, ?it/s]

In [ ]:
display(pd.DataFrame(outputs).head(1))

,context,question,answer,source_doc
0,"`f1`: the average [F1 score](https://huggingface.co/metrics/f1), on a scale between 0.0 and 1.0.\n\n\n## Examples \n\n```python\n>>> predictions = [['INTJ', 'ADP', 'PROPN', 'NOUN', 'PUNCT', 'INTJ', 'ADP', 'PROPN', 'VERB', 'SYM']]\n>>> references = [['INTJ', 'ADP', 'PROPN', 'PROPN', 'PUNCT', 'INTJ', 'ADP', 'PROPN', 'PROPN', 'SYM']]\n>>> poseval = evaluate.load(""poseval"")\n>>> results = poseval.compute(predictions=predictions, references=references)\n>>> print(list(results.keys()))\n['ADP', 'INTJ', 'NOUN', 'PROPN', 'PUNCT', 'SYM', 'VERB', 'accuracy', 'macro avg', 'weighted avg']\n>>> print(results[""accuracy""])\n0.8\n>>> print(results[""PROPN""][""recall""])\n0.5\n```\n\n## Limitations and bias\n\nIn contrast to [seqeval](https://github.com/chakki-works/seqeval), the poseval metric treats each token independently and computes the classification report over all concatenated sequences..\n\n\n## Citation\n\n```bibtex\n@article{scikit-learn,\n title={Scikit-learn: Machine Learning in {P}ython},\n author={Pedregosa, F. and Varoquaux, G. and Gramfort, A. and Michel, V.\n and Thirion, B. and Grisel, O. and Blondel, M. and Prettenhofer, P.\n and Weiss, R. and Dubourg, V. and Vanderplas, J. and Passos, A. and\n Cournapeau, D. and Brucher, M. and Perrot, M. and Duchesnay, E.},\n journal={Journal of Machine Learning Research},\n volume={12},\n pages={2825--2830},\n year={2011}\n}\n```\n \n## Further References \n- [README for seqeval at GitHub](https://github.com/chakki-works/seqeval)\n- [Classification report](https://scikit-learn.org/stable/modules/generated/sklearn.metrics.classification_report.html) \n- [Issues with seqeval](https://stackoverflow.com/questions/71327693/how-to-disable-seqeval-label-formatting-for-pos-tagging)",What is the average F1 score in the example?\n\n,"The average F1 score in the example is 0.8. (The value `results[""accuracy""]` equals 0.8.)",huggingface/evaluate/blob/main/metrics/poseval/README.md


In [ ]:
pd.DataFrame(outputs).to_csv('rag_sem_p2_test_dataset.csv')

### 1.3. Прежде чем использовать сгенеренные вопросы, их неплохо бы проверить

Здесь автор ссылается на статью декабря 23 года, но это все не догматы совершенно



> We thus build critique agents that will rate each question on several criteria, given in [this paper](https://huggingface.co/papers/2312.10003):
- **Groundedness:** can the question be answered from the given context?
- **Relevance:** is the question relevant to users? For instance, `"What is the date when transformers 4.29.1 was released?"` is not relevant for ML practitioners.

> One last failure case we've noticed is when a function is tailored for the particular setting where the question was generated, but undecipherable by itself, like `"What is the name of the function used in this guide?"`.
We also build a critique agent for this criteria:
- **Stand-alone**: is the question understandable free of any context, for someone with domain knowledge/Internet access? The opposite of this would be `What is the function used in this article?` for a question generated from a specific blog article.

> We systematically score functions with all these agents, and whenever the score is too low for any one of the agents, we eliminate the question from our eval dataset.

>  💡 ___When asking the agents to output a score, we first ask them to produce its rationale. This will help us verify scores, but most importantly, asking it to first output rationale gives the model more tokens to think and elaborate an answer before summarizing it into a single score token.___


Чистить будем агентами-критиками

In [ ]:
question_groundedness_critique_prompt = """
You will be given a context and a question.
Your task is to provide a 'total rating' scoring how well one can answer the given question unambiguously with the given context.
Give your answer on a scale of 1 to 5, where 1 means that the question is not answerable at all given the context, and 5 means that the question is clearly and unambiguously answerable with the context.

Provide your answer as follows:

Answer:::
Evaluation: (your rationale for the rating, as a text)
Total rating: (your rating, as a number between 1 and 5)

You MUST provide values for 'Evaluation:' and 'Total rating:' in your answer.

Now here are the question and context.

Question: {question}\n
Context: {context}\n
Answer::: """

question_relevance_critique_prompt = """
You will be given a question.
Your task is to provide a 'total rating' representing how useful this question can be to machine learning developers building NLP applications with the Hugging Face ecosystem.
Give your answer on a scale of 1 to 5, where 1 means that the question is not useful at all, and 5 means that the question is extremely useful.

Provide your answer as follows:

Answer:::
Evaluation: (your rationale for the rating, as a text)
Total rating: (your rating, as a number between 1 and 5)

You MUST provide values for 'Evaluation:' and 'Total rating:' in your answer.

Now here is the question.

Question: {question}\n
Answer::: """

question_standalone_critique_prompt = """
You will be given a question.
Your task is to provide a 'total rating' representing how context-independent this question is.
Give your answer on a scale of 1 to 5, where 1 means that the question depends on additional information to be understood, and 5 means that the question makes sense by itself.
For instance, if the question refers to a particular setting, like 'in the context' or 'in the document', the rating must be 1.
The questions can contain obscure technical nouns or acronyms like Gradio, Hub, Hugging Face or Space and still be a 5: it must simply be clear to an operator with access to documentation what the question is about.

For instance, "What is the name of the checkpoint from which the ViT model is imported?" should receive a 1, since there is an implicit mention of a context, thus the question is not independent from the context.

Provide your answer as follows:

Answer:::
Evaluation: (your rationale for the rating, as a text)
Total rating: (your rating, as a number between 1 and 5)

You MUST provide values for 'Evaluation:' and 'Total rating:' in your answer.

Now here is the question.

Question: {question}\n
Answer::: """

In [ ]:
print("Generating critique for each QA couple...")
for output in tqdm(outputs):
    evaluations = {
        "groundedness": call_llm(
            llm_client,
            question_groundedness_critique_prompt.format(
                context=output["context"], question=output["question"]
            ),
        ),
        "relevance": call_llm(
            llm_client,
            question_relevance_critique_prompt.format(question=output["question"]),
        ),
        "standalone": call_llm(
            llm_client,
            question_standalone_critique_prompt.format(question=output["question"]),
        ),
    }
    try:
        for criterion, evaluation in evaluations.items():
            score, eval = (
                int(evaluation.split("Total rating: ")[-1].strip()),
                evaluation.split("Total rating: ")[-2].split("Evaluation: ")[1],
            )
            output.update(
                {
                    f"{criterion}_score": score,
                    f"{criterion}_eval": eval,
                }
            )
    except Exception as e:
        continue

Generating critique for each QA couple...


  0%|          | 0/10 [00:00<?, ?it/s]

Попробуем отфильтровать "плохие вопросы":

In [ ]:
import pandas as pd

pd.set_option("display.max_colwidth", None)

generated_questions = pd.DataFrame.from_dict(outputs)

print("Evaluation dataset before filtering:")
display(
    generated_questions[
        [
            "question",
            "answer",
            "groundedness_score",
            "relevance_score",
            "standalone_score",
        ]
    ]
)
generated_questions = generated_questions.loc[
    (generated_questions["groundedness_score"] >= 4)
    & (generated_questions["relevance_score"] >= 4)
    & (generated_questions["standalone_score"] >= 4)
]
print("============================================")
print("Final evaluation dataset:")
display(
    generated_questions[
        [
            "question",
            "answer",
            "groundedness_score",
            "relevance_score",
            "standalone_score",
        ]
    ]
)

eval_dataset = datasets.Dataset.from_pandas(
    generated_questions, split="train", preserve_index=False
)

Evaluation dataset before filtering:


,question,answer,groundedness_score,relevance_score,standalone_score
0,What is the average F1 score in the example?\n\n,"The average F1 score in the example is 0.8. (The value `results[""accuracy""]` equals 0.8.)",1.0,NaN,NaN
1,How to modify the `pad_token_id` value of a given LLM model?\n,You can manually set the `pad_token_id` value for a given LLM model by calling `model.config.pad_token_id = <new value>` in your code.\n</Tip>,3.0,NaN,NaN
2,How are the examples prepared for training with TensorFlow in the context?\n,"The examples are prepared with albumentations for augmentation and wrapped in `~transformers.TFPreTrainedModel.prepare_tf_dataset` method from 🤗 Transformers, creating a `tf.data.Dataset` with collated and batched samples ready for training without the need for further modifications.",5.0,3.0,2.0
3,What type of objective does SqueezeBERT rely on?\n,SqueezeBERT relies on the masked language modeling (MLM) objective.,5.0,NaN,NaN
4,How many training episodes does the agent train for in the reinforcement learning task?\n,"The agent trains for ```pixelcopter_hyperparameters[""n_training_episodes""]``` training episodes in the reinforcement learning task.",1.0,1.0,5.0
5,What is the top 1 accuracy of mobilenetv3?\n,The top 1 accuracy of mobilenetv3 is 75.62%.,NaN,NaN,NaN
6,What is Top 1 Accuracy for dla46x_c on ImageNet?\n,The Top 1 Accuracy for dla46x_c on ImageNet is 65.98%.,5.0,1.0,5.0
7,What events should be subscribed to in order to react to data changes?\n,"""Repo update"" events should be subscribed to in order to react to data changes.",2.0,NaN,NaN
8,What should be the number of input channels while creating a UNet2DModel?\n,"The number of input channels while creating a UNet2DModel should be 3, which is the number of channels in an RGB image.",NaN,NaN,NaN
9,Where was Paper A Time Series is Worth 64 Words published?\n,The paper A Time Series is Worth 64 Words was published.,1.0,1.0,5.0


Final evaluation dataset:


,question,answer,groundedness_score,relevance_score,standalone_score


Вроде готово, но из экономии нагенерили тоже всего несколько пар

В итоге просто загрузим готовый тестовый датасет:

In [ ]:
eval_dataset = datasets.load_dataset("m-ric/huggingface_doc_qa_eval", split="train")

README.md:   0%|          | 0.00/893 [00:00<?, ?B/s]

train-00000-of-00001.parquet:   0%|          | 0.00/289k [00:00<?, ?B/s]

Generating train split:   0%|          | 0/65 [00:00<?, ? examples/s]

# 2. Теперь построим RAG и оценим что улучшает его качество

### 2.1. Наполняем базу знаний -- это мы уже умеем

Здесь автор ноутбука выделяет способы разбиения текста на [чанки](https://github.com/FullStackRetrieval-com/RetrievalTutorials/blob/main/tutorials/LevelsOfTextSplitting/5_Levels_Of_Text_Splitting.ipynb) или [чанки](https://huggingface.co/spaces/m-ric/chunk_visualizer) (намекая какой вкуснее)
- каждые `n` слов / символов
- по границам предложений, но не менее `n`  слов / символов
- **recursive split** tries to preserve even more of the document structure, by processing it tree-like way, splitting first on the largest units (chapters) then recursively splitting on smaller units (paragraphs, sentences).

Ниже как раз RecursiveCharacterTextSplitter из langchain

о проблеме в ragatouille известно с 22 года и ничего не меняется
 https://github.com/huggingface/transformers/issues/36954, исходный ноутбук февраля 24го года

In [ ]:
!pip install -q --no-cache-dir torch transformers==4.40 langchain sentence-transformers tqdm openpyxl openai pandas datasets langchain-community ragatouille==0.0.9 peft==0.10.0

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 137.6/137.6 kB 7.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.0/9.0 MB 37.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.7/45.7 kB 36.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 199.1/199.1 kB 195.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 194.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 196.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 197.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 235.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 108.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 109.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 102.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 234.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [ ]:
from ragatouille import RAGPretrainedModel

In [ ]:
#
from huggingface_hub import notebook_login

notebook_login()

In [ ]:
from tqdm.auto import tqdm
import pandas as pd
from typing import Optional, List, Tuple
import json
import datasets

pd.set_option("display.max_colwidth", None)

In [ ]:
# из части 1
eval_dataset = datasets.load_dataset("m-ric/huggingface_doc_qa_eval", split="train")

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


README.md:   0%|          | 0.00/893 [00:00<?, ?B/s]

train-00000-of-00001.parquet:   0%|          | 0.00/289k [00:00<?, ?B/s]

Generating train split:   0%|          | 0/65 [00:00<?, ? examples/s]

In [ ]:
# import tqdm
from langchain.docstore.document import Document as LangchainDocument

ds = datasets.load_dataset("m-ric/huggingface_doc", split="train")

RAW_KNOWLEDGE_BASE = [
    LangchainDocument(page_content=doc["text"], metadata={"source": doc["source"]})
    for doc in tqdm(ds)
]

  0%|          | 0/2647 [00:00<?, ?it/s]

In [ ]:
from langchain.text_splitter import RecursiveCharacterTextSplitter
from transformers import AutoTokenizer

def split_documents(
    chunk_size: int,
    knowledge_base: List[LangchainDocument],
    tokenizer_name: str,
) -> List[LangchainDocument]:
    """
    Split documents into chunks of size `chunk_size` characters and return a list of documents.
    """
    text_splitter = RecursiveCharacterTextSplitter.from_huggingface_tokenizer(
        AutoTokenizer.from_pretrained(tokenizer_name),
        chunk_size=chunk_size,
        chunk_overlap=int(chunk_size / 10),
        add_start_index=True,
        strip_whitespace=True,
        separators=["\n\n", "\n", ".", " ", ""],
    )

    docs_processed = []
    for doc in knowledge_base:
        docs_processed += text_splitter.split_documents([doc])

    # Remove duplicates
    unique_texts = {}
    docs_processed_unique = []
    for doc in docs_processed:
        if doc.page_content not in unique_texts:
            unique_texts[doc.page_content] = True
            docs_processed_unique.append(doc)

    return docs_processed_unique

### 2.2. Retriever - на базе индексов FAISS

Что можно потюнить:
*   размеры чанков
*   методы разбиения -- в тч попробуем [semantic chunking](https://python.langchain.com/docs/modules/data_connection/document_transformers/semantic-chunker)...
*   выбор модели-эмбеддер

In [ ]:
from langchain.vectorstores import FAISS
from langchain_community.embeddings import HuggingFaceEmbeddings
from langchain_community.vectorstores.utils import DistanceStrategy
import os


def load_embeddings(
    langchain_docs: List[LangchainDocument],
    chunk_size: int,
    embedding_model_name: Optional[str] = "thenlper/gte-small",
) -> FAISS:
    """
    Creates a FAISS index from the given embedding model and documents. Loads the index directly if it already exists.

    Args:
        langchain_docs: list of documents
        chunk_size: size of the chunks to split the documents into
        embedding_model_name: name of the embedding model to use

    Returns:
        FAISS index
    """
    # load embedding_model
    embedding_model = HuggingFaceEmbeddings(
        model_name=embedding_model_name,
        multi_process=True,
        model_kwargs={"device": "cuda"},
        encode_kwargs={
            "normalize_embeddings": True
        },  # set True to compute cosine similarity
    )

    # Check if embeddings already exist on disk
    index_name = (
        f"index_chunk:{chunk_size}_embeddings:{embedding_model_name.replace('/', '~')}"
    )
    index_folder_path = f"./data/indexes/{index_name}/"
    if os.path.isdir(index_folder_path):
        return FAISS.load_local(
            index_folder_path,
            embedding_model,
            distance_strategy=DistanceStrategy.COSINE,
            allow_dangerous_deserialization=True # fix для семинара
        )

    else:
        print("Index not found, generating it...")
        docs_processed = split_documents(
            chunk_size,
            langchain_docs,
            embedding_model_name,
        )
        knowledge_index = FAISS.from_documents(
            docs_processed, embedding_model, distance_strategy=DistanceStrategy.COSINE
        )
        knowledge_index.save_local(index_folder_path)
        return knowledge_index

### 2.3. LLM 💬

Здесь пробуем:
- С реранжированием и без
- Выбор модели

In [ ]:
RAG_PROMPT_TEMPLATE = """
<|system|>
Using the information contained in the context,
give a comprehensive answer to the question.
Respond only to the question asked, response should be concise and relevant to the question.
Provide the number of the source document when relevant.
If the answer cannot be deduced from the context, do not give an answer.</s>
<|user|>
Context:
{context}
---
Now here is the question you need to answer.

Question: {question}
</s>
<|assistant|>
"""

In [ ]:
from langchain_community.llms import HuggingFaceHub

## здесь у меня закончились лимиты на бесплатный HF API :)

# repo_id = "HuggingFaceH4/zephyr-7b-beta"
# READER_MODEL_NAME = "zephyr-7b-beta"
# HF_API_TOKEN = ""

# READER_LLM = HuggingFaceHub(
#     repo_id=repo_id,
#     task="text-generation",
#     huggingfacehub_api_token=HF_API_TOKEN,
#     model_kwargs={
#         "max_new_tokens": 512,
#         "top_k": 30,
#         "temperature": 0.1,
#         "repetition_penalty": 1.03,
#     },
# )

# HfHubHTTPError: 402 Client Error: Payment Required for url: https://router.huggingface.co/hf-inference/models/HuggingFaceH4/zephyr-7b-beta (Request ID: Root=1-67f81053-4d449f946643b57907b6bc15;01db4f48-ef73-44b4-a38d-f07e684e498a)

# You have exceeded your monthly included credits for Inference Providers. Subscribe to PRO to get 20x more monthly included credits.



In [ ]:
from langchain.chat_models import ChatOpenAI

READER_MODEL_NAME = "gpt-4" # дешевле использовать mini
OPENAI_API_KEY = "YOUR_TOKEN"

READER_LLM = ChatOpenAI(
    model_name=READER_MODEL_NAME,
    temperature=0.1,
    max_tokens=512,
    openai_api_key=OPENAI_API_KEY,
)

In [ ]:
from ragatouille import RAGPretrainedModel
from langchain_core.vectorstores import VectorStore
from langchain_core.language_models.llms import LLM
from langchain_core.messages import HumanMessage

def answer_with_rag(
    question: str,
    llm: LLM,
    knowledge_index: VectorStore,
    reranker: Optional[RAGPretrainedModel] = None,
    num_retrieved_docs: int = 30,
    num_docs_final: int = 7,
) -> Tuple[str, List[LangchainDocument]]:
    """Answer a question using RAG with the given knowledge index."""
    # Gather documents with retriever
    relevant_docs = knowledge_index.similarity_search(
        query=question, k=num_retrieved_docs
    )
    relevant_docs = [doc.page_content for doc in relevant_docs]  # keep only the text

    # Optionally rerank results
    if reranker:
        relevant_docs = reranker.rerank(question, relevant_docs, k=num_docs_final)
        relevant_docs = [doc["content"] for doc in relevant_docs]

    relevant_docs = relevant_docs[:num_docs_final]

    # Build the final prompt
    context = "\nExtracted documents:\n"
    context += "".join(
        [f"Document {str(i)}:::\n" + doc for i, doc in enumerate(relevant_docs)]
    )

    final_prompt = RAG_PROMPT_TEMPLATE.format(question=question, context=context)

    # Redact an answer
    # answer = llm(final_prompt) это если HF
    answer = llm([HumanMessage(content=final_prompt)])

    return answer, relevant_docs

# 3. LLM as a judge с тестовым датасетом

[Метрик для оценки RAG много](https://docs.ragas.io/en/latest/concepts/metrics/index.html), здесь же выбрана faithfulness как end-to-end метрика всей системы.

Вслед за автором:
> We use GPT4 as a judge for its empirically good performance, but you could try with other models such as [kaist-ai/prometheus-13b-v1.0](https://huggingface.co/kaist-ai/prometheus-13b-v1.0) or [BAAI/JudgeLM-33B-v1.0](https://huggingface.co/BAAI/JudgeLM-33B-v1.0).

Для GPT4 понадобятся open_api_key и включенный vpn

Промт за основу берем отсюда [Prometheus's prompt template](https://huggingface.co/kaist-ai/prometheus-13b-v1.0) -- с идеей четко прописать за что какая оценка.

Здесь мы снова вступаем в конфликт с жадностью -- просим обоснование -> больше токенов

In [ ]:
from langchain_core.language_models import BaseChatModel
from langchain_core.messages import AIMessage, HumanMessage, SystemMessage

def run_rag_tests(
    eval_dataset: datasets.Dataset,
    llm,
    knowledge_index: VectorStore,
    output_file: str,
    reranker: Optional[RAGPretrainedModel] = None,
    verbose: Optional[bool] = True,
    test_settings: Optional[str] = None,  # To document the test settings used
):
    """Runs RAG tests on the given dataset and saves the results to the given output file."""
    try:  # load previous generations if they exist
        with open(output_file, "r") as f:
            outputs = json.load(f)
    except:
        outputs = []

    for example in tqdm(eval_dataset):
        question = example["question"]
        if question in [output["question"] for output in outputs]:
            continue

        answer, relevant_docs = answer_with_rag(
            question, llm, knowledge_index, reranker=reranker
        )
        if verbose:
            print("=======================================================")
            print(f"Question: {question}")
            print(f"Answer: {answer}")
            print(f'True answer: {example["answer"]}')
        result = {
            "question": question,
            "true_answer": example["answer"],
            "source_doc": example["source_doc"],
            "generated_answer": answer,
            "retrieved_docs": [doc for doc in relevant_docs],
        }
        if test_settings:
            result["test_settings"] = test_settings
        outputs.append(result)

        def serialize_message(msg):
            if isinstance(msg, (AIMessage, HumanMessage, SystemMessage)):
                return msg.content
            return msg  # for strings or already-serializable types

        serializable_outputs = []
        for entry in outputs:
            serializable_entry = {k: serialize_message(v) for k, v in entry.items()}
            serializable_outputs.append(serializable_entry)

        # Now write to file safely
        with open(output_file, "w") as f:
            json.dump(serializable_outputs, f, indent=2)

        # with open(output_file, "w") as f:
        #     json.dump(outputs, f)

In [ ]:
EVALUATION_PROMPT = """###Task Description:
An instruction (might include an Input inside it), a response to evaluate, a reference answer that gets a score of 5, and a score rubric representing a evaluation criteria are given.
1. Write a detailed feedback that assess the quality of the response strictly based on the given score rubric, not evaluating in general.
2. After writing a feedback, write a score that is an integer between 1 and 5. You should refer to the score rubric.
3. The output format should look as follows: \"Feedback: {{write a feedback for criteria}} [RESULT] {{an integer number between 1 and 5}}\"
4. Please do not generate any other opening, closing, and explanations. Be sure to include [RESULT] in your output.

###The instruction to evaluate:
{instruction}

###Response to evaluate:
{response}

###Reference Answer (Score 5):
{reference_answer}

###Score Rubrics:
[Is the response correct, accurate, and factual based on the reference answer?]
Score 1: The response is completely incorrect, inaccurate, and/or not factual.
Score 2: The response is mostly incorrect, inaccurate, and/or not factual.
Score 3: The response is somewhat correct, accurate, and/or factual.
Score 4: The response is mostly correct, accurate, and factual.
Score 5: The response is completely correct, accurate, and factual.

###Feedback:"""

from langchain.prompts.chat import (
    ChatPromptTemplate,
    HumanMessagePromptTemplate,
)
from langchain.schema import SystemMessage


evaluation_prompt_template = ChatPromptTemplate.from_messages(
    [
        SystemMessage(content="You are a fair evaluator language model."),
        HumanMessagePromptTemplate.from_template(EVALUATION_PROMPT),
    ]
)

In [ ]:
from langchain.chat_models import ChatOpenAI
from langchain_core.messages import AIMessage, HumanMessage, SystemMessage

OPENAI_API_KEY = "YOUR_KEY"

eval_chat_model = ChatOpenAI(model="gpt-4-1106-preview", temperature=0, openai_api_key=OPENAI_API_KEY)
evaluator_name = "GPT4"


def evaluate_answers(
    answer_path: str,
    eval_chat_model,
    evaluator_name: str,
    evaluation_prompt_template: ChatPromptTemplate,
) -> None:
    """Evaluates generated answers. Modifies the given answer file in place for better checkpointing."""
    answers = []
    if os.path.isfile(answer_path):  # load previous generations if they exist
        answers = json.load(open(answer_path, "r"))

    for experiment in tqdm(answers):
        if f"eval_score_{evaluator_name}" in experiment:
            continue

        eval_prompt = evaluation_prompt_template.format_messages(
            instruction=experiment["question"],
            response=experiment["generated_answer"],
            reference_answer=experiment["true_answer"],
        )
        eval_result = eval_chat_model.invoke(eval_prompt)
        feedback, score = [
            item.strip() for item in eval_result.content.split("[RESULT]")
        ]
        experiment[f"eval_score_{evaluator_name}"] = score
        experiment[f"eval_feedback_{evaluator_name}"] = feedback

        # with open(answer_path, "w") as f:
        #     json.dump(answers, f)

        def serialize_message(msg):
            if isinstance(msg, (AIMessage, HumanMessage, SystemMessage)):
                return msg.content
            return msg  # for strings or already-serializable types

        serializable_outputs = []
        for entry in answers:
            serializable_entry = {k: serialize_message(v) for k, v in entry.items()}
            serializable_outputs.append(serializable_entry)

        # Now write to file safely
        with open(answer_path, "w") as f:
            json.dump(serializable_outputs, f, indent=2)

Попробуем

In [ ]:
if not os.path.exists("./output"):
    os.mkdir("./output")

# кто не любит трехэтажный мат -- itertools.product()
for chunk_size in [200]:  # Add other chunk sizes (in tokens) as needed
    for embeddings in ["thenlper/gte-small"]:  # Add other embeddings as needed
        for rerank in [True, False]:
            settings_name = f"chunk:{chunk_size}_embeddings:{embeddings.replace('/', '~')}_rerank:{rerank}_reader-model:{READER_MODEL_NAME}"
            output_file_name = f"./output/rag_{settings_name}.json"

            print(f"Running evaluation for {settings_name}:")

            print("Loading knowledge base embeddings...")
            knowledge_index = load_embeddings(
                RAW_KNOWLEDGE_BASE,
                chunk_size=chunk_size,
                embedding_model_name=embeddings
                )

            print("Running RAG...")
            reranker = (
                RAGPretrainedModel.from_pretrained("colbert-ir/colbertv2.0")
                if rerank
                else None
            )
            run_rag_tests(
                eval_dataset=eval_dataset,
                llm=READER_LLM,
                knowledge_index=knowledge_index,
                output_file=output_file_name,
                reranker=reranker,
                verbose=False,
                test_settings=settings_name,
            )

            print("Running evaluation...")
            evaluate_answers(
                output_file_name,
                eval_chat_model,
                evaluator_name,
                evaluation_prompt_template,
            )

Running evaluation for chunk:200_embeddings:thenlper~gte-small_rerank:True_reader-model:gpt-4:
Loading knowledge base embeddings...


/usr/local/lib/python3.11/dist-packages/huggingface_hub/file_download.py:896: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


Running RAG...


/usr/local/lib/python3.11/dist-packages/colbert/utils/amp.py:12: FutureWarning: `torch.cuda.amp.GradScaler(args...)` is deprecated. Please use `torch.amp.GradScaler('cuda', args...)` instead.
  self.scaler = torch.cuda.amp.GradScaler()


  0%|          | 0/65 [00:00<?, ?it/s]

...

/usr/local/lib/python3.11/dist-packages/huggingface_hub/file_download.py:896: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


Running RAG...


  0%|          | 0/65 [00:00<?, ?it/s]

Running evaluation...


  0%|          | 0/65 [00:00<?, ?it/s]

### Глянем что получилось

In [ ]:
import glob

outputs = []
for file in glob.glob("./output/*.json"):
    output = pd.DataFrame(json.load(open(file, "r")))
    output["settings"] = file
    outputs.append(output)
result = pd.concat(outputs)

In [ ]:
result.to_csv('RAG_2p2_result.csv')

In [ ]:
outputs

[                                                                                                               question  \
 0                                           What architecture is the `tokenizers-linux-x64-musl` binary designed for?\n   
 1                                                                    What is the purpose of the BLIP-Diffusion model?\n   
 2                                                 How can a user claim authorship of a paper on the Hugging Face Hub?\n   
 3                                        What is the purpose of the /healthcheck endpoint in the Datasets server API?\n   
 4                                    What is the default context window size for Local Attention in the LongT5 model?\n   
 ..                                                                                                                  ...   
 60  What is the maximum size of a model checkpoint before it is automatically sharded in Transformers version 4.18.0?\n   
 61     

In [ ]:
result["eval_score_GPT4"] = result["eval_score_GPT4"].apply(
    lambda x: int(x) if isinstance(x, str) else 1
)
result["eval_score_GPT4"] = (result["eval_score_GPT4"] - 1) / 4

In [ ]:
average_scores = result.groupby("settings")["eval_score_GPT4"].mean()
average_scores.sort_values()

,eval_score_GPT4
settings,
./output/rag_chunk:200_embeddings:thenlper~gte-small_rerank:False_reader-model:gpt-4.json,0.842308
./output/rag_chunk:200_embeddings:thenlper~gte-small_rerank:True_reader-model:gpt-4.json,0.876923


## Примеры

Чтобы не тратить много токенов глянем что получится если в этом ноутбуке поиграть опциями, которые мы перечислили вначале.

Еще больше в [advanced_RAG](advanced_rag).


In [ ]:
import plotly.express as px

scores = datasets.load_dataset("m-ric/rag_scores_cookbook", split="train")
scores = pd.Series(scores["score"], index=scores["settings"])

README.md:   0%|          | 0.00/319 [00:00<?, ?B/s]

train-00000-of-00001.parquet:   0%|          | 0.00/1.45k [00:00<?, ?B/s]

Generating train split:   0%|          | 0/5 [00:00<?, ? examples/s]

In [ ]:
fig = px.bar(
    scores,
    color=scores,
    labels={
        "value": "Accuracy",
        "settings": "Configuration",
    },
    color_continuous_scale="bluered",
)
fig.update_layout(
    width=1000,
    height=600,
    barmode="group",
    yaxis_range=[0, 100],
    title="<b>Accuracy of different RAG configurations</b>",
    xaxis_title="RAG settings",
    font=dict(size=15),
)
fig.layout.yaxis.ticksuffix = "%"
fig.update_coloraxes(showscale=False)
fig.update_traces(texttemplate="%{y:.1f}", textposition="outside")
fig.show()

# 3. Разве можно оценивать LLMкой другую LLM?

Можно, если подобрать промпт.

А промпт подбирается так, чтобы оценка LLM была максимально близка с человеческой оценкой на 1000-1500 размеченных с перекрытиями примеров

1) Пишем промпт и даем людям как инструкцию для оценки (минимум 2 разметчика на кейс) качества генерации RAG  
2) Даем этот промпт модели-судье  
3) Измеряем корреляцию между оценками от людей и скором от LLM-судьи + измеряем дисперсию оценок людей  
4) При необходимости корректируем промпт пока результаты п3 не начинают нас устраивают  

И самом собой -- первый этап (Retrieval) оцениваем как всегда через Recall@k